In [1]:
from tqdm.auto import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, GPT2LMHeadModel

In [2]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

model = GPT2LMHeadModel.from_pretrained("/home/drdo/Caricatures/models/scan_gpt2/checkpoint-100000")

model.generation_config.max_length = 256

/home/drdo/anaconda3/envs/nlp/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
dataset = load_dataset("scan", "simple", trust_remote_code=True)

In [11]:
dataset['test'][2]

{'commands': 'look around right twice and turn left thrice',
 'actions': 'I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_LEFT I_TURN_LEFT I_TURN_LEFT'}

In [12]:
context = "look around right twice and turn left thrice"
inputs = tokenizer(context+tokenizer.sep_token, return_tensors="pt")
tokenizer.decode(model.generate(**inputs)[0], skip_special_tokens=False).replace(context, '')

TypeError: can only concatenate str (not "NoneType") to str

In [17]:
test_dataset = dataset['test']#.select(range(100))

In [19]:
count = 0
model.to("cuda")
bar = tqdm(range(len(test_dataset)))
for example in test_dataset:
    command = example['commands']
    label = example['actions']
    inputs = tokenizer(command+" <sep> ", return_tensors="pt").to("cuda")
    output = model.generate(**inputs)[0].to("cpu")
    #output = model.generate(**inputs, num_beams=5, max_new_tokens=256)[0]
    output = tokenizer.decode(output, skip_special_tokens=True).split("<sep>")[-1].strip()
    if output == label:
        count += 1
    bar.update(1)

print(count/len(test_dataset))

  0%|          | 0/4182 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

0.6661884265901482


In [ ]:
# distilgpt2 : acc = 0.6406025824964132
# distilgpt2 : acc = 0.67 -> beam search (beam size = 3)
# gpt2 : acc = 0.810856049736968, 0.88, 